In [1]:
# Importação dos pacotes python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

In [32]:
pd.set_option("mode.copy_on_write", True)

In [33]:
# configurar os plots do matplotlib
rcParams['figure.figsize'] = 12, 4 # tamanho da grade da figura a ser criada
rcParams['lines.linewidth'] = 3 # quantidade de linhas
rcParams['xtick.labelsize'] = 'x-large' # largura do eixo x é o total da tela disponível
rcParams['ytick.labelsize'] = 'x-large' # largura do eixo y é o total da tela disponível

In [34]:
#Carregando o arquivo
arquivo = "Dados/dados_compras.json"
base = pd.read_json(arquivo, orient = "records")

### EDA 1 - Análise Geral de Compras

* Número total de consumidores
* Preço médio de compra
* Número total de compras
* Rendimento total

In [35]:
dados_demo = base[["Login","Sexo","Valor"]] # retornar todas as linhas das colunas selecionadas
dados_demo = dados_demo.drop_duplicates("Login")# retira todas os usuário que tiveram mais de uma compra

total_usuários = dados_demo["Login"].count()# Total de usuários da base demográfica
quantidade_vendas = base["Valor"].count()# Quantidade de vendas realizadas
media_vendas = base["Valor"].mean()# Médidas das vendas realizadas
total_vendas = base["Valor"].sum()# Somatório das vendas realizadas

#criar um dataframe para juntar todas as variáveis criadas 
vendas = pd.DataFrame({"Numero Total de Usuários":total_usuários,
                      "Quantidade de Vendas":quantidade_vendas,
                      "Valor Médio de Vendas": media_vendas,
                      "Total de vendas":total_vendas},
                       index= ["Total"])

#ajustar formato dos numeros com função map (só funciona com pd)
vendas["Numero Total de Usuários"] = vendas["Numero Total de Usuários"].map("{:,}".format)
vendas["Quantidade de Vendas"] = vendas["Quantidade de Vendas"].map("{:}".format)
vendas["Valor Médio de Vendas"] = vendas["Valor Médio de Vendas"].map("R$ {:,.2f}".format)
vendas["Total de vendas"] = vendas["Total de vendas"].map("R$ {:,.2f}".format)

display(vendas)

,Numero Total de Usuários,Quantidade de Vendas,Valor Médio de Vendas,Total de vendas
Total,573,780,R$ 2.93,"R$ 2,286.33"


In [ ]:
# plot



### EDA 2 - Análise Demográfica por Gênero

* Porcentagem e contagem de compradores masculinos
* Porcentagem e contagem de compradores do sexo feminino
* Porcentagem e contagem de outros / não divulgados

In [36]:
# Localizar todas as linhas das colunas sexo e valor agrupa-las por sexo
vendas_sexo = pd.DataFrame(base[["Sexo","Valor"]].groupby("Sexo").count())
#realizar a porcentagem da variável anterior
vendas_sexo_por = pd.DataFrame(base[["Sexo","Valor"]].groupby("Sexo").count()/total_vendas*100)
#Realizar a média das vendas por gênero
vendas_sexo_me = base.loc[:,["Sexo","Valor"]].groupby("Sexo").mean()

# Concatenar valores em um único Dataframe
novas_vendas = pd.concat(
                        [vendas_sexo,vendas_sexo_por,vendas_sexo_me], 
                        axis= 1, 
                        keys= ["Total de Vendas", "Porcetagem das Vendas", "Media de vendas"])
# axis = 1 indica que o dataframe será concatenado em colunas
# Keys para definir o nome dos títulos

# Formatação dos valores
# Todo ajuste de formatação dos valores
# Todo: testar o assign para criar colunas personalizadas de Porcentagem e Média
# Todo: Retirar Valor e Sexo do Dataframe
# Todo: Alinhar o texto do dataframe

display(novas_vendas)


,Total de Vendas,Porcetagem das Vendas,Media de vendas
,Valor,Valor,Valor
Sexo,,,
Feminino,136,5.948398,2.815515
Masculino,633,27.686292,2.950521
Outro / Não Divulgado,11,0.481120,3.249091


In [ ]:
leg = ["Feminino","Masculino","Outros/Não Divulgados"]
val = [133, 633, 11]

fig, ax = plt.subplots()
bar_colors = ['tab:red', 'tab:blue', 'tab:orange'] # colors
bar_container = ax.bar(leg, val, color = bar_colors)
ax.set(ylabel='Total de Vendas', title='Total por Gênero', ylim=(0, 700))
ax.bar_label(bar_container, fmt='{:,.0f}')
# todo - colocar a porcentagem no gráfico

### EDA 3 - Análise de Compras Por Gênero

* Número de compras
* Preço médio de compra
* Valor Total de Compra
* Compras for faixa etária

In [37]:
# Restringir a base de Dados para facilitar a análise
base_genero = base[["Sexo","Valor"]]

#agrupar valores por genero com o groupyby
total_gen = base_genero.groupby("Sexo").sum()
total_qntd = base_genero.groupby("Sexo").count()
total_gen_media = base_genero.groupby("Sexo").mean()
total_gen_por = base_genero.groupby("Sexo").sum()/ total_vendas*100

# Concatenar todas as tabelas em um único Dataframe
genero = pd.concat([total_gen,total_qntd,total_gen_media,total_gen_por], 
                   keys= ["Total de Vendas por Gênero", "Quantidade de Vendas", "Media de Vendas", "Porcentagem"], 
                   axis = 1)

#genero.assign()
# Todo: testar o assign para criar colunas personalizadas
# Todo: Retirar Valor e Sexo do Dataframe
# Todo: Alinhar o texto do dataframe

display(genero)

,Total de Vendas por Gênero,Quantidade de Vendas,Media de Vendas,Porcentagem
,Valor,Valor,Valor,Valor
Sexo,,,,
Feminino,382.91,136,2.815515,16.747801
Masculino,1867.68,633,2.950521,81.688995
Outro / Não Divulgado,35.74,11,3.249091,1.563204


In [ ]:
# Plot
# data from https://allisonhorst.github.io/palmerpenguins/

species = ("Feminino", "Masculino", "Outro / Não Divulgado")
penguin_means = {
    'Bill Depth': (18.35, 18.43, 14.98),
    'Bill Length': (38.79, 48.83, 47.50),
    'Flipper Length': (189.95, 195.82, 217.19),
}

x = np.arange(len(species))  # the label locations
width = 0.25  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(layout='constrained')

for attribute, measurement in penguin_means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Length (mm)')
ax.set_title('Penguin attributes by species')
ax.set_xticks(x + width, species)
ax.legend(loc='upper left', ncols=3)
ax.set_ylim(0, 250)

plt.show()

In [ ]:
base

In [38]:
# adicionando nova coluna para separar idades
base['Faixa Idade'] = np.where((base['Idade'] > 1) & (base['Idade'] <= 20), "1-20",
                np.where((base['Idade'] > 20) & (base['Idade'] <= 40), "21-40",
                np.where((base['Idade'] > 40) & (base['Idade'] <= 60), "41-60",
                np.where((base['Idade'] > 60) & (base['Idade'] <= 80), "61-80",
                np.where((base['Idade'] > 80) & (base['Idade'] <= 100), "80-100", "Acima de 100")))))

In [39]:
# Plot
base_idade = base[["Faixa Idade","Valor"]]

total_id = base_idade.groupby("Faixa Idade").sum();
total_qntd_id = base_idade.groupby("Faixa Idade").count()
total_gen_media_id = base_idade.groupby("Faixa Idade").mean()
total_gen_id = base_idade.groupby("Faixa Idade").sum()/ total_vendas*100

idade = pd.concat([total_id,total_qntd_id,total_gen_media_id,total_gen_id], 
                   keys= ["Total de Vendas por Faixa Etária", "Quantidade de Vendas", "Media de Vendas", "Porcentagem"], 
                   axis = 1)

display(idade)

,Total de Vendas por Faixa Etária,Quantidade de Vendas,Media de Vendas,Porcentagem
,Valor,Valor,Valor,Valor
Faixa Idade,,,,
1-20,849.51,294,2.889490,37.156054
21-40,1428.18,483,2.956894,62.466048
41-60,8.64,3,2.880000,0.377898


#### EDA 5 -  Consumidores Mais Populares (Top 5)
### Identifique os 5 principais compradores pelo valor total de compra e, em seguida, liste (em uma tabela): **

* Login
* Número de compras
* Preço médio de compra
* Valor Total de Compra
* Itens mais populares

In [40]:
# Restringir a base de Dados para facilitar a análise
base_consumidores = base[["Login","Valor","Nome do Item"]]

# Agrupar o total de vendas realizado por cada consumidor, ordenandos na ordem decresente do valor total de vendas
top5_tot = pd.DataFrame(base_consumidores.groupby("Login").sum()).sort_values(by = "Valor", ascending= False)
top5_cont = pd.DataFrame(base_consumidores.groupby("Login")["Valor"].count())
top5_med = pd.DataFrame(base_consumidores.groupby("Login").mean())
top_item = pd.DataFrame(base_consumidores.groupby("Login")["Nome do Item"])


# Contenar dataframes
top5 = pd.concat([top5_tot,top5_cont,top5_med,top_item], axis = 1, keys= ["Valor Total", "Numero de Compras", "Preço Médio de Compra","Item mais Comprado"])

# Todo: Como retornar o nomme do item vendido
# Todo: testar o assign para criar colunas personalizadas

# Demonstrar apenas os 5 primeiros valores de dataframe
display(top5.head(5))

/tmp/ipykernel_1803/3459582330.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  top5_tot = pd.DataFrame(base_consumidores.groupby("Login").sum()).sort_values(by = "Valor", ascending= False)
/tmp/ipykernel_1803/3459582330.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  top5_med = pd.DataFrame(base_consumidores.groupby("Login").mean())


Valor Total Numero de Compras Preço Médio de Compra  \
                  Valor             Valor                 Valor   
Undirrala66       17.06               5.0              3.412000   
Saedue76          13.56               4.0              3.390000   
Mindimnya67       12.74               4.0              3.185000   
Haellysu29        12.73               3.0              4.243333   
Eoda93            11.58               3.0              3.860000   

            Item mais Comprado       
                             0    1  
Undirrala66                NaN  NaN  
Saedue76                   NaN  NaN  
Mindimnya67                NaN  NaN  
Haellysu29                 NaN  NaN  
Eoda93                     NaN  NaN

In [ ]:
# Plot

### EDA 6 - Itens Mais Populares
### Identifique os 5 itens mais populares por contagem de compras e, em seguida, liste (em uma tabela):

* ID do item
* Nome do item
* Número de compras
* Preço do item
* Valor Total de Compra
* Itens mais lucrativos

In [ ]:
# Restringir a base de Dados para facilitar a análise
base_itens = base.loc[["Item ID","Nome do Item", "Valor"]]
display(base_itens)

### EDA 7 - Itens Mais Lucrativos
### Identifique os 5 itens mais lucrativos pelo valor total de compra e, em seguida, liste (em uma tabela): **

* ID do item
* Nome do item
* Número de compras
* Preço do item
* Valor Total de Compra

In [ ]:
# Restringir a base de Dados para facilitar a análise

In [ ]:
# agrupar os items mais lucrativo
item = pd.DataFrame(base_itens.groupby("Nome do Item").sum("Valor")).sort_values(by= "Valor",ascending=False)

# 

display(item.head(5))

ML 1 - Engenharia de Atributo